In [13]:
%load_ext autoreload
%autoreload 1
import tensorflow as tf
import gensim
import pyspark 
import pyspark.sql.functions as fn
%aimport ihop.community2vec
%aimport ihop.import_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
spark = ihop.import_data.get_spark_session()

Spark configuration:
[('spark.driver.port', '34833'), ('spark.executor.id', 'driver'), ('spark.app.id', 'local-1641512392312'), ('spark.driver.host', '192.168.0.11'), ('spark.sql.warehouse.dir', 'file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/notebooks/spark-warehouse'), ('spark.app.name', 'IHOP import data'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.app.startTime', '1641512390850'), ('spark.executor.memory', '4G'), ('spark.ui.showConsoleProgress', 'true')]


In [15]:
context_df = spark.read.csv("../data/community2vec/sample_data/user_contexts").select(fn.col("_c0").alias("subreddit"))
context_df.show()

+--------------------+
|           subreddit|
+--------------------+
|              cigars|
|          edc_raffle|
|             Calgary|
|              spacex|
|           AskReddit|
|            greenday|
|              boston|
|           pokemongo|
|         TheaterDays|
|       SquaredCircle|
|           Stellaris|
|     DBZDokkanBattle|
|The_Donald The_Do...|
|          Beekeeping|
|      AboveandBeyond|
|                 nba|
|       psychotherapy|
|        youtubehaiku|
|Anarcho_Capitalis...|
|                 PS4|
+--------------------+
only showing top 20 rows



In [16]:
# TODO Does data need to be randomized?

df_split = context_df.select(fn.split("subreddit"," ").alias('subreddit_list')).select("subreddit_list", fn.size("subreddit_list").alias("num_comments"))
max_comments = df_split.agg(fn.max("num_comments")).head()[0]
print(max_comments)
df_split.show()

258
+--------------------+------------+
|      subreddit_list|num_comments|
+--------------------+------------+
|            [cigars]|           1|
|        [edc_raffle]|           1|
|           [Calgary]|           1|
|            [spacex]|           1|
|         [AskReddit]|           1|
|          [greenday]|           1|
|            [boston]|           1|
|         [pokemongo]|           1|
|       [TheaterDays]|           1|
|     [SquaredCircle]|           1|
|         [Stellaris]|           1|
|   [DBZDokkanBattle]|           1|
|[The_Donald, The_...|           2|
|        [Beekeeping]|           1|
|    [AboveandBeyond]|           1|
|               [nba]|           1|
|     [psychotherapy]|           1|
|      [youtubehaiku]|           1|
|[Anarcho_Capitali...|           2|
|               [PS4]|           1|
+--------------------+------------+
only showing top 20 rows



In [17]:
def subreddit_generator(context_df):
	df_iter = context_df.select(context_df.subreddit).toLocalIterator()
	for row in df_iter:
		yield row.subreddit.split()
	

In [18]:
vocab = ihop.community2vec.get_vocabulary("../data/community2vec/sample_data/subreddit_counts.csv")
print("VOCAB SIZE:", len(vocab))
print("VOCAB SNIPPET:")
for i, (k,v) in enumerate(vocab.items()):
    print(k,":",v)
    if i>10:
        break


VOCAB SIZE: 2275
VOCAB SNIPPET:
AskReddit : 486
CFB : 403
CrazyIdeas : 261
news : 158
ConciseIAmA : 147
4chan4trump : 136
politics : 117
RocketLeagueExchange : 96
The_Donald : 90
nba : 90
edc_raffle : 77
worldnews : 74


In [19]:
num_users = context_df.count()
print(num_users)

7231


In [32]:
c2v_model = gensim.models.word2vec.Word2Vec(vector_size=150, window=max_comments, sg=1, hs=0, negative=35, compute_loss=True, epochs=1000)
c2v_model.build_vocab_from_freq(vocab)
#print("Random initialization")
#print(c2v_model.wv.most_similar("The_Donald", topn=10))
#c2v_model.train(, total_examples=num_users, epochs=1000, compute_loss=True)

In [33]:
c2v_model.wv.most_similar("The_Donald", topn=10)

KeyError: "Key 'The_Donald' not present"

In [2]:
c2v_dataset = tf.data.experimental.make_csv_dataset(
    file_pattern="../data/community2vec/test/user_contexts/*.csv",
    batch_size=1024,
    column_names=["user","subreddits"],
    header=False
    #shuffle=True,
    #shuffle_seed=100
)
# TODO Use map & reduce to find longest element in datset for max window
print(c2v_dataset)

<PrefetchDataset shapes: OrderedDict([(user, (1024,)), (subreddits, (1024,))]), types: OrderedDict([(user, tf.string), (subreddits, tf.string)])>


2022-01-06 16:49:09.463036: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-06 16:49:09.463061: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-06 16:49:09.463079: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Kurt): /proc/driver/nvidia/version does not exist
2022-01-06 16:49:09.463395: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
for i, r in enumerate(c2v_dataset):
    if i%1000==0:
        print(r['user'], r['subreddits'])
print(i)

tf.Tensor(
[b'bluestaplerman' b'okay_reasons' b'TeaInUS' ... b'Hanspanzer'
 b'HS_Highruleking' b'Kyrion530'], shape=(1024,), dtype=string) tf.Tensor(
[b'CFB CFB CFB' b'ApplyingToCollege'
 b'darkjokes ClashRoyale darkjokes ClashRoyale' ... b'leagueoflegends'
 b'Battlejack' b'bravefrontier'], shape=(1024,), dtype=string)
tf.Tensor(
[b'Readbooks6' b'ACalz' b'burgerbroyo' ... b'kabekew' b'Jerbo_Da_Klerb'
 b'globaltourist'], shape=(1024,), dtype=string) tf.Tensor(
[b'exmormon exmormon exmormon' b'leafs' b'dankchristianmemes' ...
 b'flightsim' b'smashbros' b'gaming'], shape=(1024,), dtype=string)
tf.Tensor(
[b'Sirinox' b'Attendat' b'afraidoflamp' ... b'zvive' b'RNorth2'
 b'dreezyforsheezy'], shape=(1024,), dtype=string) tf.Tensor(
[b'RimWorld' b'de' b'AskReddit AskReddit' ... b'exmormon' b'history'
 b'The_Keepers The_Keepers'], shape=(1024,), dtype=string)
tf.Tensor(
[b'StringTailor' b'Dalek1234' b'SoaresSkeleton93' ... b'ilikefries'
 b'lolmusic0954' b'drooooooooo'], shape=(1024,), dtype=str

In [6]:
# TODO determine max window size
# TODO Build model for pre-processing data, including negative sampling and skipgrams
# TODO Efficiently iterate through batches

text_vectorizer = tf.keras.layers.TextVectorization(standardize=None, output_mode='int', vocabulary=vocab)
preproc_model = tf.keras.models.Sequential()
preproc_model.add(text_vectorizer)

# TODO batch iteration
#transformed_data = preproc_model.predict([r['subreddits'] for r in c2v_dataset])


In [9]:
# TODO Build model for training W2v
text_vectorizer.get_vocabulary()

['AskReddit',
 'CFB',
 'CrazyIdeas',
 'news',
 'ConciseIAmA',
 '4chan4trump',
 'politics',
 'RocketLeagueExchange',
 'The_Donald',
 'nba',
 'edc_raffle',
 'worldnews',
 'funny',
 'teenagers',
 'leagueoflegends',
 'Ice_Poseidon',
 'DestinyTheGame',
 'pics',
 'hockey',
 'nfl',
 'gaming',
 'baseball',
 'videos',
 'Jokes',
 'Fireteams',
 'SquaredCircle',
 'rugbyunion',
 'anime',
 'pcmasterrace',
 'relationships',
 'LifeProTips',
 'gifs',
 'PoliticalHumor',
 'MLS',
 'fantasyfootball',
 'DotA2',
 'Rainbow6',
 'Showerthoughts',
 'MMA',
 'PUBATTLEGROUNDS',
 'mildlyinteresting',
 'exmormon',
 'europe',
 'gonewild',
 'FireEmblemHeroes',
 'movies',
 'neoliberal',
 'Overwatch',
 'RocketLeague',
 'aww',
 'todayilearned',
 'GlobalOffensive',
 'rupaulsdragrace',
 'FIFA',
 'NBA2k',
 'conspiracy',
 '2007scape',
 'PS4',
 'btc',
 'cars',
 'hiphopheads',
 'soccer',
 'ChapoTrapHouse',
 'FFBraveExvius',
 'MaddenUltimateTeam',
 'ffxiv',
 'Bitcoin',
 'CringeAnarchy',
 'Guildwars2',
 'CasualConversation',
 'Co